In [61]:
import numpy as np
import pandas as pd

import statsmodels.api as sm
import statsmodels.stats.api as sms
import pylab as py
import scipy.linalg as la
import statistics
import scipy.stats as stats
import scipy

from math import gamma as tma
import itertools
from scipy.stats import laplace
from scipy.stats import logistic
from scipy.stats import cauchy
from scipy.stats import binom
from scipy.stats import weibull_min as weibull
from scipy.stats import poisson
from scipy.stats import gamma
from scipy.stats import beta
from scipy.stats import norm
from scipy.stats import multivariate_normal as mnorm
from scipy.stats import t as studt
from scipy.stats import f as fdist
from scipy.stats import chisquare as chisq
from scipy.stats import chi2
from scipy.stats import gaussian_kde as gkde
from sklearn.neighbors import KernelDensity
import math

import seaborn as sns

import matplotlib.pyplot as plt
from matplotlib.cbook import boxplot_stats

import warnings
warnings.filterwarnings('ignore')

#### Exercise 4.9.1
Consider the sulfur dioxide concentrations data discussed in Example $4.1.3$. Use the R function *percentciboot* to obtain a bootstrap $95\%$ confidence interval for the true mean concentration. Use $3000$ bootstraps and compare it with the t-confidence interval for the mean.

In [62]:
data = pd.read_csv('data/sulfurdio.csv')
data.head()

,Unnamed: 0,sulfurdioxide
0,1,33.4
1,2,38.6
2,3,41.7
3,4,43.9
4,5,44.4


The website *https://www.codeconvert.ai/r-to-python-converter* was used to convert the R function to Python function below.

In [63]:
def percentciboot(x, b, alpha):
    theta = np.mean(x)
    thetastar = np.zeros(b)
    n = len(x)
    for i in range(b):
        xstar = np.random.choice(x, n, replace=True)
        thetastar[i] = np.mean(xstar)
    thetastar = np.sort(thetastar)
    pick = round((alpha/2) * (b+1))
    lower = thetastar[pick]
    upper = thetastar[b-pick+1]
    return {'theta': theta, 'lower': lower, 'upper': upper, 'thetasta': thetastar}

In [64]:
n = 3000
a = 0.05
percentciboot(data['sulfurdioxide'],n,a)

{'theta': 53.916666666666664,
 'lower': 49.90416666666667,
 'upper': 57.770833333333336,
 'thetasta': array([46.40833333, 47.1125    , 47.4875    , ..., 60.30416667,
        60.825     , 61.32083333])}

In [65]:
def dxt(a,df,t):
    return (studt.cdf(a, df, loc=0, scale=1)-t)
def invert_studt(t,df):
    a = 10.0 * int(np.sqrt(df/(df-2)))
    b = -10.0 * int(np.sqrt(df/(df-2)))
    c = (a+b)/2
    tol = 0.00001

    while(abs(dxt(c,df,t)) > tol):
        c = (a+b)/2
        if(dxt(c,df,t) > 0):
            a = c
        else:
            b = c
    return c

In [66]:
xb = np.mean(data['sulfurdioxide'])
nb = len(data['sulfurdioxide'])
s = np.std(data['sulfurdioxide'],ddof=1)
tn = invert_studt(1-a/2,nb-1)

print(xb-tn*s/np.sqrt(nb),xb+tn*s/np.sqrt(nb))

49.663272631181506 58.17006070215182


#### Exercise 4.9.2

Let $x_1,x_2,...,x_n$ be the values of a random sample. A bootstrap sample, $x^{∗′} = (x_1^*,x_2^*,...,x_n^*)$, is a random sample of $x_1,x_2,...,x_n$ drawn with replacement.

(a) Show that $x_1^*,x_2^*,...,x_n^*$ are iid with common cdf $\hat{F}_n$􏰦 , the empirical cdf of $x_1,x_2,...,x_n$.

(b) Show that $E(x_i^* ) = \bar{x}$.

(c) If n is odd, show that median ${x_i^*} = x_{((n+1)/2)}$.

(d) Show that $V (x_i^*) = n^{−1} \sum􏰋_{i=1}^n (x_i − \bar{x})^2$.


Solution is in the Solutions Manual and is marked as exercise 4.9.1 while it is actually 4.9.2.

#### Exercise 4.9.3

Let $X_1, X_2, ... , X_n$ be a random sample from a $\Gamma(1, \beta)$ distribution.

(a) Show that the confidence interval $(2n\bar{X}/(\chi_{2n}^2)^{(1−(\alpha/2))}, 2n\bar{X}/(\chi_{2n}^2)^{(\alpha/2)})$ is an exact $(1 − \alpha)100\%$ confidence interval for $\beta$.

(b) Using part $(a)$, show that the $90\%$ confidence interval for the data of Example $4.9.1$ is $(64.99, 136.69)$.

(a) If $X_1, X_2, ... , X_n$ are random samples from a $\Gamma(1, \beta)$ distribution, and $\bar{X}$ is their mean, then $n\bar{X} = \sum X_i$ is distributed as $\Gamma(n,\beta)$. This then means $2n\bar{X}/\beta$ is distributed as $\Gamma(n,2)$ which is our pivot random variable with a $\chi_{2n}^2$ distribution.

Just call $a = (\chi_{2n}^2)^{(\alpha/2)}$, the $\alpha/2$ significance level for the Chi-square distribution and likewise, define $b = (\chi_{2n}^2)^{(1-\alpha/2)}$. This makes the expressions less cumbersome!!

Then we have $a < 2n\bar{X}/\beta < b$ which is same as $2n\bar{X}/b < \beta < 2n\bar{X}/a$, ie the desired interval is $(2n\bar{X}/b,2n\bar{X}/a)$.

In [67]:
# (b)

samples = [131.7,182.7,73.3,10.7,150.4,42.3,22.2,17.9,264.0,154.4,4.3,265.6,61.9,10.8,48.8,22.5,8.8,150.6,103.0,85.9]
n = len(samples)
xb = np.mean(samples)

In [68]:
def dxc2(a,t,dof):
    return (chi2.cdf(a, dof)-t)
def invert_chi2(t,dof):
    a = 0
    b = 100.0
    c = (a+b)/2
    tol = 0.00001

    while(abs(dxc2(c,t,dof)) > tol):
        c = (a+b)/2
        if(dxc2(c,t,dof) < 0):
            a = c
        else:
            b = c
    return c

In [69]:
t5 = invert_chi2(0.05,2*n)
t95 = invert_chi2(0.95,2*n)

# Answer
print((2*n*xb/t95,2*n*xb/t5))

(64.98720639264407, 136.6927126000115)


#### Exercise 4.9.4

Consider the situation discussed in Example $4.9.1$. Suppose we want to estimate the median of $X_i$ using the sample median.

(a) Determine the median for a $\Gamma(1,\beta)$ distribution.

(b) The algorithm for the bootstrap percentile confidence intervals is general and hence can be used for the median. Rewrite the R code in the function percentciboot.s so that the median is the estimator. Using the sample given in the example, obtain a $90%\$ bootstrap percentile confidence interval for the median. Did it trap the true median in this case?

(a) Solution given in solutions manual (as solution of 4.9.3 but it is actually for 4.9.4). With a strictly increasing cdf, it is very easy to show that median is $\beta \log{(2)}$.

In [70]:
# (b)

def percentciboot_median(x, b, alpha):
    theta = np.median(x)
    thetastar = np.zeros(b)
    n = len(x)
    for i in range(b):
        xstar = np.random.choice(x, n, replace=True)
        thetastar[i] = np.median(xstar)
    thetastar = np.sort(thetastar)
    pick = round((alpha/2) * (b+1))
    lower = thetastar[pick]
    upper = thetastar[b-pick+1]
    return {'theta': theta, 'lower': lower, 'upper': upper, 'thetasta': thetastar}

In [71]:
samples = [131.7,182.7,73.3,10.7,150.4,42.3,22.2,17.9,264.0,154.4,4.3,265.6,61.9,10.8,48.8,22.5,8.8,150.6,103.0,85.9]
nb = 3000
a = 0.1
percentciboot_median(samples,nb,a)

{'theta': 67.6,
 'lower': 22.5,
 'upper': 131.7,
 'thetasta': array([  9.75,  14.35,  16.5 , ..., 154.4 , 154.4 , 166.65])}

#### Exercise 4.9.5

Not sure what is needed to be shown other than reconicling the change in notation.

#### Exercise 4.9.6

In [72]:
# (b) Took this code from solutions manual and converted to Python online (link given above)

def bootstrap_standardized(x, b, alpha):
    theta = np.mean(x)
    stan = np.sqrt(np.var(x))
    n = len(x)
    teestar = np.zeros(b)
    for i in range(b):
        xstar = np.random.choice(x, n, replace=True)
        teestar[i] = (np.mean(xstar) - theta) / (np.sqrt(np.var(xstar)) / np.sqrt(n))
    teestar = np.sort(teestar)
    pick = round((alpha / 2) * (b + 1))
    lower0 = teestar[pick]
    upper0 = teestar[b - pick + 1]
    lower = theta - upper0 * (stan / np.sqrt(n))
    upper = theta - lower0 * (stan / np.sqrt(n))
    return {'theta': theta, 'lower': lower, 'upper': upper, 'teestar': teestar}

In [73]:
sample = [119.7,104.1,92.8,85.4,108.6,93.4,67.1,88.4,101.0,97.2,95.4,77.2,100.0,114.2,150.3,102.3,105.8,107.5,0.9,94.1]
nb = 3000
a = 0.1

In [74]:
bootstrap_standardized(samples,nb,a)

{'theta': 90.58999999999999,
 'lower': 62.34514494805455,
 'upper': 128.72796461945882,
 'teestar': array([-7.36077712, -5.22474816, -4.78831576, ...,  3.27636606,
         3.43430779,  3.53466547])}

#### Exercise 4.9.7

In [75]:
# (b)

def boottesttwo_median(x, y, b):
    n1 = len(x)
    n2 = len(y)
    v = np.median(y)-np.median(x)
    z = y + x
#     print(z)
    counter = 0
    teststatall = [0] * b
    for i in range(b):
        xstar = np.random.choice(z, n1, replace=True)
        ystar = np.random.choice(z, n2, replace=True)
        vstar = np.median(ystar)-np.median(xstar)
#         print(xstar)
#         print(ystar)
        if vstar >= v:
            counter += 1
        teststatall[i] = vstar
    pvalue = counter / b
    return {'origtest': v, 'pvalue': pvalue, 'teststatall': teststatall}

In [76]:
# (c)

xa = [94.2,111.3,90.0,99.7,116.8,92.2,166.0,95.7,109.3,106.0,111.7,111.9,111.6,146.4,103.9]
yb = [125.5,107.1,67.9,98.2,128.6,123.5,116.5,143.2,120.3,118.6,105.0,111.8,129.3,130.8,139.8]

nb = 3000

boottesttwo_median(xa,yb,nb)

{'origtest': 11.0,
 'pvalue': 0.06866666666666667,
 'teststatall': [-2.200000000000003,
  -0.29999999999999716,
  -8.399999999999991,
  5.200000000000003,
  -8.399999999999991,
  -2.6000000000000085,
  -9.0,
  -21.099999999999994,
  -1.7000000000000028,
  16.39999999999999,
  0.0,
  3.299999999999997,
  0.10000000000000853,
  5.799999999999997,
  4.599999999999994,
  4.8999999999999915,
  -4.800000000000011,
  5.200000000000003,
  9.0,
  8.599999999999994,
  -0.30000000000001137,
  -12.200000000000003,
  13.599999999999994,
  4.800000000000011,
  -7.200000000000003,
  -7.5,
  6.900000000000006,
  4.900000000000006,
  -4.6000000000000085,
  -13.200000000000003,
  7.299999999999997,
  8.599999999999994,
  0.0,
  4.700000000000003,
  -5.799999999999997,
  6.799999999999997,
  -2.5,
  4.700000000000003,
  -4.8999999999999915,
  4.599999999999994,
  -3.299999999999997,
  4.8999999999999915,
  0.5,
  -4.6000000000000085,
  -7.8999999999999915,
  11.700000000000003,
  -5.200000000000003,
  -4

#### Exercise 4.9.8

Consider the data of Example $4.9.2$. The two-sample t-test of Example $4.6.2$ can be used to test these hypotheses. The test is not exact here (why?), but it is an approximate test. Show that the value of the test statistic is $t = 0.93$, with an approximate p-value of $0.18$.

In [77]:
def dxt(a,df,t):
    return (studt.cdf(a, df, loc=0, scale=1)-t)
def invert_studt(t,df):
    a = 10.0 * int(np.sqrt(df/(df-2)))
    b = -10.0 * int(np.sqrt(df/(df-2)))
    c = (a+b)/2
    tol = 0.00001

    while(abs(dxt(c,df,t)) > tol):
        c = (a+b)/2
        if(dxt(c,df,t) > 0):
            a = c
        else:
            b = c
    return c

In [78]:
xx = [94.2,111.3,90.0,99.7,116.8,92.2,166.0,95.7,109.3,106.0,111.7,111.9,111.6,146.4,103.9]
yy = [125.5,107.1,67.9,98.2,128.6,123.5,116.5,143.2,120.3,118.6,105.0,111.8,129.3,130.8,139.8]

xb = np.mean(xx)
sx = np.var(xx,ddof=1)
n1 = len(xx)

yb = np.mean(yy)
sy = np.var(yy,ddof=1)
n2 = len(yy)

n = n1 + n2

sp = ((n1-1)*sx + (n2-1)*sy)/(n-2)

t = (yb-xb)/np.sqrt(sp/n1+sp/n2)
p = 1-studt.cdf(t, n-2, loc=0)
print(t,p)

0.9298320806470268 0.1802028640897948


#### Exercise 4.9.9

In [79]:
samples = [119.7,104.1,92.8,85.4,108.6,93.4,67.1,88.4,101.0,97.2,95.4,77.2,100.0,114.2,150.3,102.3,105.8,107.5,0.9,94.1]

In [80]:
def boottestonemean(x, mu0, b):
    n = len(x)
    v = np.mean(x)
    z = x - np.mean(x) + mu0
    counter = 0
    teststatall = np.zeros(b)
    
    for i in range(b):
        xstar = np.random.choice(z, n, replace=True)
        vstar = np.mean(xstar)
        if np.abs(vstar) >= np.abs(v):
            counter += 1
        teststatall[i] = vstar
    
    pvalue = counter / b
    return {'origtest': v, 'pvalue': pvalue, 'teststatall': teststatall}

In [81]:
nb = 3000
mu0 = 90
boottestonemean(samples,mu0,nb)

{'origtest': 95.27,
 'pvalue': 0.19433333333333333,
 'teststatall': array([91.29 , 78.415, 85.195, ..., 97.905, 86.035, 89.165])}

#### Exercise 4.9.10

In [82]:
# (a)

xp = [10,15,21]
yp = [20,25,30]

zp = xp+yp
v = np.mean(yp) - np.mean(xp)

combs = list(itertools.combinations(zp,3))

m = len(combs) 
mp = 0
for xi in combs:
    yi = list(set(zp)-set(xi))
    vi = np.mean(yi) - np.mean(xi)
    if(vi >= v):
        mp = mp+1
p = mp/m
print(p,v,m)

0.1 9.666666666666666 20


In [83]:
# (b)
def boottesttwo_permutation(x, y, b):
    n1 = len(x)
    n2 = len(y)
    v = np.mean(y)-np.mean(x)
    z = y + x
    counter = 0
    teststatall = [0] * b
    for i in range(b):
        xstar = np.random.choice(z, n1, replace=False)
        ystar = list(set(z)-set(xstar))
        vstar = np.mean(ystar)-np.mean(xstar)
#         print(len(xstar))
#         print(len(ystar))
#         print(len(z))
        if vstar >= v:
            counter += 1
        teststatall[i] = vstar
    pvalue = counter / b
    return {'origtest': v, 'pvalue': pvalue, 'teststatall': teststatall}

In [84]:
xa = [94.2,111.3,90.0,99.7,116.8,92.2,166.0,95.7,109.3,106.0,111.7,111.9,111.6,146.4,103.9]
yb = [125.5,107.1,67.9,98.2,128.6,123.5,116.5,143.2,120.3,118.6,105.0,111.8,129.3,130.8,139.8]

nb = 3000

boottesttwo_permutation(xa,yb,nb)

{'origtest': 6.626666666666651,
 'pvalue': 0.181,
 'teststatall': [-4.000000000000014,
  -6.013333333333321,
  -1.0133333333333496,
  -3.346666666666664,
  4.439999999999998,
  -3.386666666666656,
  -9.333333333333314,
  4.013333333333335,
  -1.6399999999999864,
  1.6666666666666572,
  -0.5333333333333314,
  3.906666666666638,
  -1.2399999999999807,
  7.2266666666666595,
  7.493333333333325,
  1.5333333333333172,
  -3.773333333333298,
  -0.18666666666666742,
  -1.2000000000000028,
  6.839999999999989,
  3.3200000000000074,
  -6.346666666666664,
  1.1200000000000188,
  -5.786666666666662,
  -4.026666666666671,
  9.333333333333343,
  10.933333333333337,
  10.053333333333342,
  3.1333333333333258,
  1.13333333333334,
  -2.6000000000000227,
  -5.2533333333333445,
  -1.2533333333333303,
  -0.86666666666666,
  4.720000000000013,
  1.5199999999999818,
  -1.5333333333333314,
  3.40000000000002,
  -3.5733333333333093,
  -0.5466666666666669,
  7.533333333333331,
  -2.0799999999999983,
  -0.25333

(c) There are $n!$ permutations that result in distinct samples, and there are $n^n$ possible samples . Hence the probability is $n!/n^n$

#### Exercise 4.9.11

Solution given in solutions manual as the solution to 4.9.10 but it is actually solution for 4.9.11

#### Exercise 4.9.12

In [85]:
sample = [119.7,104.1,92.8,85.4,108.6,93.4,67.1,88.4,101.0,97.2,95.4,77.2,100.0,114.2,150.3,102.3,105.8,107.5,0.9,94.1]

xb = np.mean(sample)
n = len(sample)
mu = 90
sx=np.std(sample,ddof=1)

t = (xb-mu)/(sx/np.sqrt(n))
p = 1-studt.cdf(t, n-1, loc=0)
print(t,p)

0.8447507282007096 0.20438081802273078


#### Exercise 4.9.13

In [86]:
def boottestonemed(x, theta0, b):
    n = len(x)
    v = np.median(x)
    z = x - np.median(x) + theta0
    counter = 0
    teststatall = np.zeros(b)
    for i in range(b):
        xstar = np.random.choice(z, n, replace=True)
        vstar = np.median(xstar)
        if vstar >= v:
            counter += 1
        teststatall[i] = vstar
    pvalue = counter / b
    return {'origtest': v, 'pvalue': pvalue, 'teststatall': teststatall}

In [87]:
sample = [119.7,104.1,92.8,85.4,108.6,93.4,67.1,88.4,101.0,97.2,95.4,77.2,100.0,114.2,150.3,102.3,105.8,107.5,0.9,94.1]
nb = 3000
a = 0.1
med0 = 90

boottestonemed(samples,med0,nb)

{'origtest': 98.6,
 'pvalue': 0.004666666666666667,
 'teststatall': array([88.6, 90.5, 84.2, ..., 91.9, 88.6, 88.6])}

#### Exercise 4.9.14

In [94]:
def pairsbootstrap(x, y, nb):
    d = x - y - np.mean(x) + np.mean(y)
    n = len(d)
    ts = np.mean(x) - np.mean(y)
    tsstar = np.zeros(nb)
    pval = 0
    
    for i in range(nb):
        dstar = np.random.choice(d, n, replace=True)
        tsstar[i] = np.mean(dstar)
        if tsstar[i] >= ts:
            pval += 1
    
    pval = pval / nb
    
    return {'teststat': ts, 'pval': pval, 'tsstar': tsstar}

In [96]:
data = pd.read_csv('data/darwin.csv')
nb = 10000
pairsbootstrap(data['cross'],data['self'], nb)

{'teststat': 2.620000000000001,
 'pval': 0.0068,
 'tsstar': array([ 0.51333333, -1.58666667, -0.395     , ...,  1.21333333,
         0.055     ,  0.65833333])}